In [1]:
from collections.abc import Sequence
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import json
import numpy as np
import sklearn.feature_extraction.text as sk_text
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import os
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    b = plt.plot(t['pred'].tolist(),label='prediction')
    a = plt.plot(t['y'].tolist(),label='expected')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low

def read_from_json_file(file_path, fields_to_extract):
    data = []
    counter = 0
    limit = 100000
    progress = 100000
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            row = json.loads(line)
            extracted_data = {}
            
            
            for field in fields_to_extract:
                if field in row:
                    extracted_data[field] = row[field]
                    
            data.append(extracted_data)
            counter += 1

            #if counter % limit == 0:
             #   break
            #if counter % progress == 0:
             #   print(str(counter) + " lines processed...")

    return data

In [2]:
start = time.time()

business_json = 'yelp_academic_dataset_business.json'
review_json = 'yelp_academic_dataset_review.json'

#Feature 1 (Include Categories in Data Frame)
business_field = ['business_id', 'stars', 'review_count', 'categories', 'city']
business_data = read_from_json_file(business_json, business_field)
business_df = pd.DataFrame(business_data)
business_df = business_df.rename(columns={'stars': 'business_star'})

review_field = ['business_id', 'stars', 'text']
review_data = read_from_json_file(review_json, review_field)
review_df = pd.DataFrame(review_data)
review_df = review_df.rename(columns={'stars': 'review_star'})

business_df.head()

,business_id,business_star,review_count,categories,city
0,Pns2l4eNsfO8kk83dixA6A,5.0,7,"Doctors, Traditional Chinese Medicine, Naturop...",Santa Barbara
1,mpf3x-BjTdTEA3yCZrAYPw,3.0,15,"Shipping Centers, Local Services, Notaries, Ma...",Affton
2,tUFrWirKiKi_TAnsVWINQQ,3.5,22,"Department Stores, Shopping, Fashion, Home & G...",Tucson
3,MTSW4McQd7CbVtyjqoe9mw,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Philadelphia
4,mWMc6_wTdE0EUBKIGXDVfA,4.5,13,"Brewpubs, Breweries, Food",Green Lane


In [3]:
review_df.head()

,business_id,review_star,text
0,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,"If you decide to eat here, just be aware it is..."
1,7ATYjTIgM3jUlt4UM3IypQ,5.0,I've taken a lot of spin classes over the year...
2,YjUWPpI6HXG530lwP-fb2A,3.0,Family diner. Had the buffet. Eclectic assortm...
3,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favo..."
4,e4Vwtrqf-wpJfwesgvdgxQ,4.0,Cute interior and owner (?) gave us tour of up...


In [4]:
df_stars = review_df.groupby('business_id')['review_star'].mean()
df_stars = df_stars.round(2)
df_stars.head()

business_id
---kPU91CF4Lq2-WlRu9Lw    4.50
--0iUa4sNDFiZFrAdIWhZQ    3.21
--30_8IhuyMHbSOcNWd6DQ    3.56
--7PUidqRWpRSpXebiyxTg    1.75
--7jw19RH9JKXgFohspgQw    4.23
Name: review_star, dtype: float64

In [5]:
#Feature 2 (Choose Minimum # of Reviews per Business)
mask = business_df['review_count'] >= 20

filtered_business_df = business_df[mask]
filtered_business_df.head()

,business_id,business_star,review_count,categories,city
2,tUFrWirKiKi_TAnsVWINQQ,3.5,22,"Department Stores, Shopping, Fashion, Home & G...",Tucson
3,MTSW4McQd7CbVtyjqoe9mw,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Philadelphia
12,il_Ro8jwPlHresjw9EGmBg,2.5,28,"American (Traditional), Restaurants, Diners, B...",Indianapolis
14,0bPLkL0QhhPO5kt1_EXmNQ,4.5,100,"Food, Delis, Italian, Bakeries, Restaurants",Largo
15,MUTTqe8uqyMdBl186RmNeA,4.0,245,"Sushi Bars, Restaurants, Japanese",Philadelphia


In [6]:
review_combined = review_df.groupby('business_id')['text'].sum()

review_dfe = pd.DataFrame({'business_id': review_combined.index, 'all_reviews':review_combined.values})

review_dfe.head()

# review_dfe.to_csv('review_combined.tsv', sep='\t', index=False)

,business_id,all_reviews
0,---kPU91CF4Lq2-WlRu9Lw,Ate here for the 1st time on Saturday 08/07/20...
1,--0iUa4sNDFiZFrAdIWhZQ,Very good San Salvadorian place ! Authentic an...
2,--30_8IhuyMHbSOcNWd6DQ,We stopped going to Action Karate in December ...
3,--7PUidqRWpRSpXebiyxTg,"This place is disgusting, and proof that Edmon..."
4,--7jw19RH9JKXgFohspgQw,"This is the best dentist in the area, hands do..."


In [7]:
merged_review_df = pd.merge(filtered_business_df, review_dfe, on='business_id', how='inner')

merged_review_df = pd.merge(merged_review_df, df_stars, on='business_id', how='inner')
#merged_review_df.to_csv('r-com.tsv', sep='\t', index=False)


In [8]:
merged_review_df.head()

,business_id,business_star,review_count,categories,city,all_reviews,review_star
0,tUFrWirKiKi_TAnsVWINQQ,3.5,22,"Department Stores, Shopping, Fashion, Home & G...",Tucson,We are fans of Target. They seem to have a li...,3.50
1,MTSW4McQd7CbVtyjqoe9mw,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Philadelphia,This is nice little Chinese bakery in the hear...,4.06
2,il_Ro8jwPlHresjw9EGmBg,2.5,28,"American (Traditional), Restaurants, Diners, B...",Indianapolis,Went there at 4am and there was only one waitr...,2.41
3,0bPLkL0QhhPO5kt1_EXmNQ,4.5,100,"Food, Delis, Italian, Bakeries, Restaurants",Largo,The worst Chicken Parm. Sandwich I've ever eat...,4.39
4,MUTTqe8uqyMdBl186RmNeA,4.0,245,"Sushi Bars, Restaurants, Japanese",Philadelphia,Stopped in to check out this new spot around t...,4.20


In [9]:
merged_review_df= pd.concat([merged_review_df.drop('categories', axis=1), merged_review_df['categories'].str.get_dummies(sep=",")], axis =1)
merged_review_df.head()


,business_id,business_star,review_count,city,all_reviews,review_star,& Probates,3D Printing,ATV Rentals/Tours,Acai Bowls,...,Wine Tasting Classes,Wine Tasting Room,Wine Tours,Wineries,Women's Clothing,Wraps,Yelp Events,Yoga,Ziplining,Zoos
0,tUFrWirKiKi_TAnsVWINQQ,3.5,22,Tucson,We are fans of Target. They seem to have a li...,3.50,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,MTSW4McQd7CbVtyjqoe9mw,4.0,80,Philadelphia,This is nice little Chinese bakery in the hear...,4.06,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,il_Ro8jwPlHresjw9EGmBg,2.5,28,Indianapolis,Went there at 4am and there was only one waitr...,2.41,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0bPLkL0QhhPO5kt1_EXmNQ,4.5,100,Largo,The worst Chicken Parm. Sandwich I've ever eat...,4.39,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MUTTqe8uqyMdBl186RmNeA,4.0,245,Philadelphia,Stopped in to check out this new spot around t...,4.20,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
merged_review_df = pd.get_dummies(merged_review_df, columns=['city'])

merged_review_df.head()

,business_id,business_star,review_count,all_reviews,review_star,& Probates,3D Printing,ATV Rentals/Tours,Acai Bowls,Accessories,...,city_kop,city_lawrence,city_meridian,city_philadelphia,city_reno,city_riverview,city_sewell,city_tampa,city_tucson,city_wilmington
0,tUFrWirKiKi_TAnsVWINQQ,3.5,22,We are fans of Target. They seem to have a li...,3.50,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,MTSW4McQd7CbVtyjqoe9mw,4.0,80,This is nice little Chinese bakery in the hear...,4.06,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,il_Ro8jwPlHresjw9EGmBg,2.5,28,Went there at 4am and there was only one waitr...,2.41,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0bPLkL0QhhPO5kt1_EXmNQ,4.5,100,The worst Chicken Parm. Sandwich I've ever eat...,4.39,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MUTTqe8uqyMdBl186RmNeA,4.0,245,Stopped in to check out this new spot around t...,4.20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
vector = sk_text.TfidfVectorizer(stop_words='english',max_features = 2500, min_df = 30, max_df = 200, token_pattern=r'\b[^\d\W_]{4,}\b')

matrix = vector.fit_transform(merged_review_df['all_reviews'])
  
print(type(matrix))
print(matrix.toarray())



<class 'scipy.sparse._csr.csr_matrix'>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
#print(vector.get_feature_names())

In [13]:
tfidf_data = matrix.toarray()

tfidf_data.shape


(61919, 2500)

In [14]:
merged_review_df = merged_review_df.drop(['all_reviews', 'business_id'], axis=1)
merged_review_df.head()

,business_star,review_count,review_star,& Probates,3D Printing,ATV Rentals/Tours,Acai Bowls,Accessories,Accountants,Acne Treatment,...,city_kop,city_lawrence,city_meridian,city_philadelphia,city_reno,city_riverview,city_sewell,city_tampa,city_tucson,city_wilmington
0,3.5,22,3.50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4.0,80,4.06,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2.5,28,2.41,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.5,100,4.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.0,245,4.20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# encode_numeric_zscore(merged_review_df, 'business_star')
encode_numeric_zscore(merged_review_df, 'review_count')
# encode_numeric_zscore(merged_review_df, 'review_star')

#merged_review_df.to_csv('r-com.tsv', sep='\t', index=False)

In [16]:
x,y = to_xy(merged_review_df, 'business_star')
x.shape

(61919, 3040)

In [17]:
y.shape

(61919,)

In [18]:
combined_df = np.concatenate((x, tfidf_data), axis=1)
combined_df.shape


(61919, 5540)

In [19]:
##### first model relu activation and adam optimizer
x_train, x_test, y_train, y_test = train_test_split(combined_df, y, test_size=0.25, random_state=100)

for i in range(5):
    
    model = Sequential()
    model.add(Dense(100, input_dim=combined_df.shape[1], activation='relu'))
    model.add(Dense(50,activation='relu'))
    model.add(Dense(25,activation='relu'))
    model.add(Dense(1, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer='adam')

    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-2, patience=3, verbose=1, mode='auto')

    checkpointer = ModelCheckpoint(filepath="dnn/best_weight_relu.hdf5", verbose=0, save_best_only=True) # save best model)

    model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor, checkpointer],verbose=0,epochs=1000)

model.load_weights('dnn/best_weight_relu.hdf5')



Epoch 4: early stopping
Epoch 6: early stopping
Epoch 4: early stopping
Epoch 4: early stopping


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential_4/dense_16/MatMul/MatMul' defined at (most recent call last):
    File "C:\Users\Moose\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Moose\anaconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Moose\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Moose\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\Moose\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\Moose\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\Moose\anaconda3\lib\asyncio\base_events.py", line 603, in run_fore

In [ ]:
predOne = model.predict(x_test)

scoreOne = np.sqrt(metrics.mean_squared_error(predOne,y_test))
print("Final score (RMSE): {}".format(scoreOne))

In [ ]:
df_y = pd.DataFrame(y_test, columns=['ground_truth'])
df_pred = pd.DataFrame(predOne, columns=['predicted'])
result = pd.concat([df_y, df_pred],axis=1)
result

In [ ]:
##### second model sigmoid activation and adam optimizer
for i in range(5):
    
    model = Sequential()
    model.add(Dense(100, input_dim=combined_df.shape[1], activation='sigmoid'))
    model.add(Dense(75,activation='sigmoid'))
    model.add(Dense(25,activation='sigmoid'))
    model.add(Dense(1, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer='adam')

    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-2, patience=3, verbose=1, mode='auto')

    checkpointer = ModelCheckpoint(filepath="dnn/best_weight_sig.hdf5", verbose=0, save_best_only=True) # save best model

    model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor, checkpointer],verbose=0,epochs=1000)

model.load_weights('dnn/best_weight_sig.hdf5')

In [ ]:
predTwo = model.predict(x_test)

scoreTwo = np.sqrt(metrics.mean_squared_error(predTwo,y_test))
print("Final score (RMSE): {}".format(scoreTwo))

In [ ]:
df_y = pd.DataFrame(y_test, columns=['ground_truth'])
df_pred = pd.DataFrame(predTwo, columns=['predicted'])
result = pd.concat([df_y, df_pred],axis=1)
result

In [ ]:
##### third model tanh activation and sgd optimizer
for i in range(5):
    
    model = Sequential()
    model.add(Dense(100, input_dim=combined_df.shape[1], activation='tanh'))
    model.add(Dense(75,activation='tanh'))
    model.add(Dense(25,activation='tanh'))
    model.add(Dense(1, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer='sgd')

    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-2, patience=3, verbose=1, mode='auto')

    checkpointer = ModelCheckpoint(filepath="dnn/best_weight_tanh.hdf5", verbose=0, save_best_only=True) # save best model

    model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor, checkpointer],verbose=0,epochs=1000)


model.load_weights('dnn/best_weight_tanh.hdf5')


In [ ]:
predThree = model.predict(x_test)
 
scoreThree = np.sqrt(metrics.mean_squared_error(predThree,y_test))
print("Final score (RMSE): {}".format(scoreThree))

In [ ]:
df_y = pd.DataFrame(y_test, columns=['ground_truth'])
df_pred = pd.DataFrame(predThree, columns=['predicted'])
result = pd.concat([df_y, df_pred],axis=1)
result

In [ ]:
RMSE_List = [scoreOne, scoreTwo, scoreThree]
bestRMSE = min(RMSE_List)

for i in range(len(RMSE_List)):
    print("Model " + str(i+1) + " RMSE was: " + str(RMSE_List[i]))

In [ ]:
if(bestRMSE == scoreOne):
    print("Model 1 was the best! RMSE was: " + str(scoreOne))
    chart_regression(predOne.flatten(),y_test, sort= True)
elif(bestRMSE == scoreTwo):
    print("Model 2 was the best! RMSE was: " + str(scoreTwo))
    chart_regression(predTwo.flatten(),y_test, sort= True)
elif(bestRMSE == scoreThree):
    print("Model 3 was the best! RMSE was: " + str(scoreThree))
    chart_regression(predThree.flatten(),y_test, sort= True)


In [ ]:
print("Model 1 Chart.")
chart_regression(predOne.flatten(),y_test, sort= True)

In [ ]:
print("Model 2 Chart.")
chart_regression(predTwo.flatten(),y_test, sort= True)

In [ ]:
end = time.time()
elapsed = end - start
print(hms_string(elapsed))